In [1]:
import numpy as np
import pandas as pd
import networkx as nx


from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
from sklearn.metrics import roc_auc_score

import statsmodels.api as sm
from statsmodels.stats.multitest import fdrcorrection
from statsmodels.formula.api import ols

import sys
sys.path.append("..")
sys.path.append("../..")
from cluster_analysis import *
# from LOR_calculation import *

import warnings

# Mtb

## Presence matrix

In [ ]:
species='Mycobacterium_tuberculosis'

genome_ids_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/genome_ids/{species}_genome_ids.csv"
clstr_freq_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}_cluster_frequencies.csv"
clstr_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta.clstr"
clstr_fasta_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta"

samples_df = pd.read_csv(genome_ids_file, dtype=str)
samples_list=samples_df['genome.genome_ids'].tolist()

clstr_freq_df=pd.read_csv(clstr_freq_file, index_col=1)
clstr_freq_df=clstr_freq_df.drop(clstr_freq_df.columns[0], axis=1)
clstr_list=clstr_freq_df.index.tolist()

df = pd.DataFrame(index=samples_list, columns=clstr_list)
df = df.fillna(0)

with open(clstr_file) as f:
    for line in f:
        if line.startswith(">Cluster"):
            #if its a > line get the cluster id and start counting its occurences in which samples by saving it to the local var
            cluster_id=line[1:].strip()
        else:
            #else its a line designating the sample genome, one that the last cluster_id is present in
            # get the 1st group matching here \d\t\d+aa, >fig\|([\d\.\d]+).+

            genome_id=re.match(r"\d+\t\d+aa, >fig\|([\d\.\d]+).+", line).group(1)
            genome_id=genome_id[:-1]  #removing the trailing .
            df.loc[genome_id, cluster_id]+=1

df = df.T
df.to_csv(f'../../data/presence_matrices/{species}_GxS.csv')

## SVM

In [ ]:
species='Mycobacterium_tuberculosis'; drug=''

presence_df = pd.read_csv(f'../../data/presence_matrices/{species}_GxS.csv', index_col=0); 
X_df=presence_df.T
X_df.index = X_df.index.astype('float')
# X_df.index = X_df.index.astype('object')


pheno_df= pd.read_csv(f'../../data/phenotypes/{species}_{drug}.csv', index_col=0)
y_df=pheno_df
y_df.index = y_df.index.astype('float')

for gene in X_df.columns:
    
    if X_df[gene].std() == 0:
        X_df.drop(gene, axis=1, inplace=True)

X_df = X_df.sort_index()
y_df = y_df.sort_index()


y_indices=list(y_df.index)
X_indices=list(X_df.index)

intersection = [i for i in y_indices if i in X_indices]
y_df = y_df.loc[intersection]
X_df = X_df.loc[intersection]

# -- removing duplicate rows? --          to check th e nature of duplicates it could be an error through type conversion
X_df = X_df.loc[~X_df.index.duplicated(keep='first')]

X = X_df.values
y = y_df.values

labeled_matrix = pd.concat([X_df, y_df], axis=1)

# E coli

In [2]:
species='Escherichia_coli'

X_df = pd.read_csv(f'../../data/temp/X_{species}_SxG_filtered_hypo_unknowns_freq5.csv', index_col=0) # filtering

## presence

In [ ]:
# species='Escherichia_coli'

# genome_ids_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/genome_ids/{species}_genome_ids.csv"
# clstr_freq_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}_cluster_frequencies.csv"
# clstr_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta.clstr"
# clstr_fasta_file=f"../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta"

# samples_df = pd.read_csv(genome_ids_file, dtype=str)
# samples_list=samples_df['genome.genome_ids'].tolist()

# clstr_freq_df=pd.read_csv(clstr_freq_file, index_col=1)
# clstr_freq_df=clstr_freq_df.drop(clstr_freq_df.columns[0], axis=1)
# clstr_list=clstr_freq_df.index.tolist()

# df = pd.DataFrame(index=samples_list, columns=clstr_list)
# df = df.fillna(0)

# with open(clstr_file) as f:
#     for line in f:
#         if line.startswith(">Cluster"):
#             #if its a > line get the cluster id and start counting its occurences in which samples by saving it to the local var
#             cluster_id=line[1:].strip()
#         else:
#             #else its a line designating the sample genome, one that the last cluster_id is present in
#             # get the 1st group matching here \d\t\d+aa, >fig\|([\d\.\d]+).+

#             genome_id=re.match(r"\d+\t\d+aa, >fig\|([\d\.\d]+).+", line).group(1)
#             genome_id=genome_id[:-1]  #removing the trailing .
#             df.loc[genome_id, cluster_id]+=1

# df = df.T
# df.to_csv(f'../../data/presence_matrices/{species}_GxS.csv')

## filtration

### unknowns and hypotheticals and freq < 5

In [ ]:
clstr_df= get_cluster_representatives(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta.clstr')
_product_df=get_representative_products(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta')
product_df = combine_cluster_product(clstr_df, _product_df)
freq_df = get_cluster_frequency(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}_cluster_frequencies.csv')

#  --counting those thar have hypotehtical in the product name
product_df['hypothetical'] = product_df['product_name'].str.contains('hypothetical', case=False)
product_df['hypothetical'].sum()

#  --same for unknown
product_df['unknown'] = product_df['product_name'].str.contains('unknown', case=False)
product_df['unknown'].sum()

# now will get a list of clusters that are either hypo or unknown to filter out
hypothetical_clusters = product_df[product_df['hypothetical']==True].index.tolist()
unknown_clusters = product_df[product_df['unknown']==True].index.tolist()

to_remove = list(set(hypothetical_clusters).union(unknown_clusters))

# -- get the list of clusters that are present in less than 5 genomes
low_freq_clusters = freq_df[freq_df['frequency']<5].index.tolist()

to_remove = list(set(to_remove).union(low_freq_clusters))

In [ ]:
presence_df = pd.read_csv(f'../../data/presence_matrices/{species}_GxS.csv', index_col=0); 
X_df=presence_df.T
X_df.index = X_df.index.astype('float')
# X_df.index = X_df.index.astype('object')

# -- removing hypothetical and unknown clusters nad those present in lesss than 5 samples
X_df = X_df.drop(to_remove, axis=1)

# -- memoized for later use if needed
X_df.to_csv(f'../../data/temp/X_{species}_SxG_filtered_hypo_unknowns_freq5.csv')


### variance threshold

In [ ]:
#  -- removing genes belowe a variance threshold
v_t=0.1

for gene in X_df.columns:
    if X_df[gene].var() < v_t: 
        X_df.drop(gene, axis=1, inplace=True)

X_df.to_csv(f'../../data/temp/X_{species}_SxG_filtered_var_{v_t}.csv')

In [ ]:
species='Escherichia_coli'; drug=''; v_t=0.1

X_df = pd.read_csv(f'../../data/temp/X_{species}_SxG_filtered_var_{v_t}.csv', index_col=0)

## labels - y

In [3]:
trimethoprim = ['folA','dfrA','dfrG']

In [25]:
drugs = ['streptomycin',
 'sulfamethoxazole',
 'tetracycline',
 'cefalothin',
 'trimethoprim_sulphamethoxazole',
 'amoxicillin_clavulanic_acid',
 'trimethoprim',
 'amoxicillin',
 'ampicillin',
 'doripenem',
 'levofloxacin',
 'ciprofloxacin']

In [3]:
drug='ciprofloxacin'

pheno_df= pd.read_csv(f'../../metadata/{species}/{species}_{drug}.csv', index_col=0)
y_df=pheno_df
y_df.index = y_df.index.astype('float')

y_df = y_df.sort_index()

y_indices=list(y_df.index)


X_df = X_df.sort_index()
y_df = y_df.sort_index()

y_indices=list(y_df.index)
X_indices=list(X_df.index)

intersection = [i for i in y_indices if i in X_indices]
y_df = y_df.loc[intersection]
X_df = X_df.loc[intersection]

X_df = X_df.sort_index()
y_df = y_df.sort_index() # -- just making sure bcs im paranoid

X = X_df.values
y = y_df.values



labeled_matrix = pd.concat([X_df, y_df], axis=1)
labeled_matrix.shape

(1672, 18876)

## Feature attributes

In [ ]:
# Dihydrofolate reductase (EC 1.5.1.3) for folA

product_df = product_df.drop(to_remove)

In [ ]:
#  -- dont forget to add this to a module (maybe cluster analysis)
def transform_cluster_to_product(cluster_name, clstr_product_df):
    #match it ffrom clstr_poduct_df from the index
    cluster = clstr_product_df.loc[cluster_name]
    product=cluster['product_name']
    
    return product

## Feature importance

Want to perform different statistical and machine learning approaches to associate each feature (gene) with the phenotype of interest (one drug at a time).  _The phenotype of interest is the resistance to a specific drug._  
Would lated take the top 100 genes of each approach and compare how 1. if the known ARGs are ranked high in each and 2. if there is enough overlap. Would probably combine all to create a pool of genes to test for interaction later on.

Ideas on statistical approaches:  
- Mutual Information (MI)
- Fisher's exact test/Chi-squared test
- ANOVA
- Jaccard index

Ideas on machine learning approaches:
- Random Forest
- SVM
- Logistic Regression
- Gradient Boosting
- Neural Networks

Would probably start with ensemble methods, using weights sum/mean as the final score.

In [ ]:
def parse_ARGs(ARG_products:pd.DataFrame, top_genes:pd.DataFrame):
    '''
    Takes in a list of ARG products and a list of top genes and returns a dict of ARGs that are in the top genes (with their rank)
    '''
    ARGs = {}
    for gene in top_genes:
        for product in ARG_products:
            if product in gene:
                ARGs[top_genes.index(gene)]=gene
    return ARGs

### MI

In [ ]:
# -- computing MI between each col and the last col in labeled_matrix
from sklearn.metrics import mutual_info_score

mi_scores = {}
for col in labeled_matrix.columns[:-1]:
    mi_scores[col]=mutual_info_score(labeled_matrix[col], labeled_matrix['SIR'])

In [ ]:
# getting top 100 scores

MI_top_100 = sorted(mi_scores, key=mi_scores.get, reverse=True)[:100]

In [ ]:
for i in  range(len(MI_top_100)):
    MI_top_100[i]=transform_cluster_to_product(MI_top_100[i], product_df)

In [ ]:
# MI_top_100

In [ ]:
# -- checking if tetramethoprim are there
trimethoprim = ['folA','dfrA','dfrG']
trimethoprim_products = ['Dihydrofolate reductase (EC 1.5.1.3)','trimethoprim-resistant dihydrofolate reductase DfrA1','Dihydrofolate reductase (EC 1.5.1.3)']

parse_ARGs(trimethoprim_products, MI_top_100)

### SVM

## Feature Interaction

Want to detect interactions between the pool of genes we now have, also following statictical and machine learning approaches.

Statistical approaches:
- 2 way ANOVA
- correlation (rho) t test (this should be coupled with a ML ensembl)

Machine learning approaches:
- Logistic Regression with interaction terms
- SVM ensemble
Following the assumption in Kavvas et al. (2018), we can use SVM and consider high correlation between features as a sign of interaction.
- Neural Networks
They mentioned that it would be nice to consider other models in other works. I think NN would be a good candidate because, even though it non-interpretable, it can capture complex relationship between features, if we can find a way to correlate the weights with the features. For this purpose, we might consider 

_For computational complexity, considering the interactions to test for significance our of the SVMs correlation_


**ALWAYS ACCOUNTING FOR MULTIPLE TESTING USING THE ENJAMINI-HOCHBERG CORRECTION**

### ANN

In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np

2024-05-01 13:47:21.649043: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 13:47:21.650456: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 13:47:21.708302: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 13:47:21.916225: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-05-01 13:47:22.718910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(200, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

#compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#train the model
model.fit(X, y, epochs=10, batch_size=32)

features = X_df.columns.to_list()
W=model.get_weights() # its an array of arrays

Epoch 1/10


/home/rayane/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5633 - loss: 2.8547
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7470 - loss: 1.3594
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8555 - loss: 0.4120
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8547 - loss: 0.4180
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9140 - loss: 0.1764
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9837 - loss: 0.0733
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9935 - loss: 0.0630
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9887 - loss: 0.0480
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9957 - loss: 0.0352
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9960 - loss: 0.0318


In [14]:
len(W[0]), len(W[1]), len(W[2]), len(W[3])
# W[0] array 18875x200 representing weights W of each feature in each of the 200 inner neurons
# W[1] array of 200x1 represents biases b from each of the neuron
# W[2] array od 200x1 representing the weights W of each of the z (neuron output in hidden layer) in the output neuron
# W[3] array 1x1 representing the bias b used in teh last output neuron

(18875, 200, 200, 1)

### SVM

In [5]:
n_models = 200


models=[]
for i in range(n_models):
    X_boot, y_boot = resample(X, y, random_state=i)
    model=SGDClassifier(loss='hinge', penalty= 'l1', max_iter=1000, tol=1e-3)
    model.fit(X_boot, y_boot.ravel())
    models.append(model)

weights=np.zeros((n_models, X.shape[1]))
for i in range(n_models):
    weights[i]=models[i].coef_
weights_df = pd.DataFrame(weights, columns=X_df.columns)

corr_SVM = weights_df.corr()

In [ ]:
# save it in a csv
corr_SVM.to_csv(f'../../data/temp/corr_SVM_{species}_{drug}.csv')

In [4]:
corr_SVM = pd.read_csv(f'../../data/temp/corr_SVM_{species}_{drug}.csv', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/temp/corr_SVM_Escherichia_coli_ciprofloxacin.csv'

In [ ]:
t = 0.1 # -- UNSIGNED

G=nx.Graph()

for i in range(X.shape[1]):

    for j in range(i+1, X.shape[1]):
        node_i=X_df.columns[i]; node_j=X_df.columns[j]
        correlation = corr_SVM.iloc[i, j]
        if correlation > t:
            G.add_edge(node_i, node_j, weight=correlation)

# G=set_cluster_attributes(G)
# nx.set_node_attributes(G, log_odds_dict, 'log_odds')

nx.write_graphml(G, f'../../data/nets/{species}/{n_models}_randomized_SVM_{t}unsigned_corr_{drug}.graphml')

edges_list = list(G.edges)
edge_weights = [G[u][v]['weight'] for u, v in edges_list]
print(f"E: {len(edges_list)}, V: {G.number_of_nodes()} when {n_models} models are used, with corr threshold={t}; species={species}, drug={drug}.")


In [ ]:
# nx.write_graphml(G, f'../../data/nets/{species}/{n_models}_randomized_SVM_{t}unsigned_corr_{drug}.graphml')

# edges_list = list(G.edges)
# edge_weights = [G[u][v]['weight'] for u, v in edges_list]
# print(f"E: {len(edges_list)}, V: {G.number_of_nodes()} when {n_models} models are used, with corr threshold={t}; species={species}, drug={drug}.")


### Logistic Regression

In [ ]:
G=nx.read_graphml('../../data/nets/Escherichia_coli/200_randomized_SVM_0.2unsigned_corr_trimethoprim.graphml')
edge_list = list(G.edges)

In [ ]:
import statsmodels.formula.api as smf

X_pairs = [(i.replace(' ', '_'), j.replace(' ', '_')) for i, j in edge_list]
interaction_labeled_matrix = labeled_matrix.rename(columns={col: col.replace(' ', '_') for col in labeled_matrix.columns})

# formula = f'SIR ~ {a} + {b} + {a}:{b}'

test_pairs = X_pairs[:1]

for pair in test_pairs:
    a, b = pair
    print(" --- looking into the edge between", a, "and", b)

    results = smf.logit(formula=f'SIR ~ {a} + {b} + {a}:{b}', data=interaction_labeled_matrix).fit()


In [ ]:
X_pairs=edge_list
error_test=[("Cluster 0", "Cluster 20")]

p_values_list = []

for pair in error_test:
    a, b = pair
    print(" --- looking into the edge between", a, "and", b)

    X_selected = X_df.loc[:, [a, b]]

    print(X_selected.shape)

    X_selected = sm.add_constant(X_selected)

    X_selected['interaction'] = X_selected.iloc[:, a] * X_selected.iloc[:, b]

    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            logit_model = sm.Logit(y, X_selected)
            result = logit_model.fit()
            p_values_list.append(result.pvalues['interaction'])

    except:
        print('singular matrix; deleted')


# reject, corrected_p_values= multipletests(p_values_list, method='fdr_bh')



In [ ]:
p_values_list

In [ ]:
result.summary()

In [ ]:
test = X_df.loc[:, ["Cluster 0", "Cluster 20"]]
# -- check if the 2 cols are identical
test = sm.add_constant(test)

    # if X_selected.shape[1]==2:
    #     X_selected['interaction'] = X_selected.iloc[:, 1] * X_selected.iloc[:, 0]
    # X_selected['interaction'] = X_selected.iloc[:, 1] * X_selected.iloc[:, 2]

test['interaction']=test.loc[:,"Cluster 0"] * test.loc[:,"Cluster 20"]
test

# X_selected
# get corr
# X_selected.corr()

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import pandas

np.random.seed(111)
df = pd.DataFrame(np.random.binomial(1,0.5,(50,3)),columns=['x1','x2','y'])

res1 = smf.logit(formula='y ~ x1 + x2 + x1:x2', data=df).fit()

res1.summary()

In [ ]:
smf.logit(formula='y ~ x1 + x2 + x1:x2', data=df).

In [ ]:
count=0; insig=0
save_edges=[]
p_values_list=[]

for pair in edges_list:
    a, b = pair
    print(" --- looking into the edge between", a, "and", b)

    X_selected = X_df.loc[:, [a, b]]

    print(X_selected.shape)

    X_selected = sm.add_constant(X_selected)
    print(X_selected.shape)
    if X_selected.shape[1]==2:
        X_selected['interaction'] = X_selected.iloc[:, 1] * X_selected.iloc[:, 0]
    X_selected['interaction'] = X_selected.iloc[:, 1] * X_selected.iloc[:, 2]

    print(X_selected.shape)
    
    col=False


    #account for collinearity
    # --- matrix way
    if np.linalg.matrix_rank(X_selected) < X_selected.shape[1]:
        print('collinearity; deleted')
        # G.remove_edge(*pair)
        continue

    # --- paper way
    # if (X_selected['interaction']==X_selected[a]).all() or (X_selected['interaction']==X_selected[b]).all():      
    #     col=True  
    #     print(f'colinearity between interaction and one of the features {a} or {b}; deleted')
    # elif (X_selected[b]==X_selected[a]).all():
    #     col=True
    #     print(f'colinearity between {a} and {b}; deleted')
    # elif (X_selected['const']==X_selected[a]).all() or (X_selected['const']==X_selected[b]).all() or (X_selected['const']==X_selected['interaction']).all():
    #     col=True
    #     print('colinearity between one of the features and the constant term?!')

    # if col:
    #     G.remove_edge(*pair)
    #     continue

    # --- logistic regression
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            logit_model = sm.Logit(y, X_selected)
            result = logit_model.fit()
    except:
        print('singular matrix; deleted')

    p_values_list.append(p_values['interaction'])

    # coefficients = result.params
    p_values = result.pvalues

    if p_values['interaction']<0.05:
        print('not significant; deleted')
        # G.remove_edge(*pair)
        insig+=1
    else:
        count+=1
        save_edges.append(pair)


    # --- multiple testing Benjamini-Hochberg correction
    reject, corrected_p_values= multipletests(p_values_list, method='fdr_bh')

    for i, pair in enumerate(edges_list):
        if corrected_p_values[i] < 0.05:
            print(f'Edge {pair} is not significant; deleted')
            G.remove_edge(pair)

### 2-way ANOVA

In [ ]:
def perform_2way_ANOVA(X_pairs, labeled_matrix):
    '''
    takes a list of tuples (pairs) - supposed to be pairs of features found in labeled_matrix - and a labeled matrix (X + y)
    returns teh significant interaction (those having corr p_val<0.05) 
    accounting for multiple testing, using the Benjamini-Hochberg FDR correction

    
    '''

In [ ]:
X_pairs = [(i.replace(' ', '_'), j.replace(' ', '_')) for i, j in edge_list]
interaction_labeled_matrix = labeled_matrix.rename(columns={col: col.replace(' ', '_') for col in labeled_matrix.columns})


In [ ]:
edges_list = list(G.edges)
edge_weights = [G[u][v]['weight'] for u, v in edges_list]
print(f"E: {len(edges_list)}, V: {G.number_of_nodes()} when {n_models} models are used, with corr threshold={t}; species={species}, drug={drug}.")

save_edges=[]

edges_list_2=[(i.replace(' ', '_'), j.replace(' ', '_')) for i, j in edges_list]
labeled_matrix_2 = labeled_matrix.rename(columns={col: col.replace(' ', '_') for col in labeled_matrix.columns})
p_values={}

count_nan=0
save_nan_testing=[]

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for pair in edges_list_2:
        a, b = pair
        # print(" --- looking into the interaction between", a, "and", b)
        formula = f'SIR ~ {a} + {b} + {a}:{b}'
        model = ols(formula, data=labeled_matrix_2).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)

        if not np.isnan(anova_table['PR(>F)'][2]):
            p_values[anova_table.index[2]]=anova_table['PR(>F)'][2]
        else:
            save_nan_testing.append(pair)
            count_nan+=1
        
print(f"{count_nan} interactions have nan p-values - removed")

# mst correction - using fdr

gene_gene_pair = list(p_values.keys())
p_values_list = list(p_values.values())

reject, corrected_p_values= fdrcorrection(p_values_list)

results_df = pd.DataFrame({
    'gene_gene': gene_gene_pair,
    'p-value': p_values_list,
    'corr_p_value': corrected_p_values,
    'reject_H0': reject
})

count_rejected=0
for i in reject:
    if i:
        count_rejected+=1

print(f"Number of rejected H0: {count_rejected} out of {len(reject)}")

### LOR of co-occurence

## testing

In [ ]:
clstr_df= get_cluster_representatives(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta.clstr')
_product_df=get_representative_products(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta')
product_df = combine_cluster_product(clstr_df, _product_df)

In [ ]:
#  left joing X_df and product_df on the index
X_t= X_df.T
test = X_t.join(product_df, how='left')
l=test['product_name'].to_list()

In [ ]:
test['Cluster'] = test.index
test.index = test['product_name']

In [ ]:
_product_df.index= _product_df['product_name']
p_ =_product_df.drop('product_name', axis=1, inplace=True)

In [ ]:
# inner join test and _product_df on the index
test = test.join(_product_df, how='inner', lsuffix='_caller', rsuffix='_other')
test

In [ ]:
test['product_name'].to_list()